## Threading with one thread

In [17]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,),daemon = True)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    x.join()
    logging.info("Main    : all done")

17:49:59: Main    : before creating thread
17:49:59: Main    : before running thread
17:49:59: Thread 1: starting
17:49:59: Main    : wait for the thread to finish
17:50:01: Thread 1: finishing
17:50:01: Main    : all done


## Threading with multiple thread(3)

In [28]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    threads = list()
    for index in range(3):
        logging.info("Main    : create and start thread %d.", index)
        x = threading.Thread(target=thread_function, args=(index,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        logging.info("Main    : before joining thread %d.", index)
        thread.join()
        logging.info("Main    : thread %d done", index)

11:10:03: Main    : create and start thread 0.
11:10:03: Thread 0: starting
11:10:03: Main    : create and start thread 1.
11:10:03: Thread 1: starting
11:10:03: Main    : create and start thread 2.
11:10:03: Thread 2: starting
11:10:03: Main    : before joining thread 0.
11:10:05: Thread 0: finishing
11:10:05: Main    : thread 0 done
11:10:05: Main    : before joining thread 1.
11:10:05: Thread 2: finishing
11:10:05: Thread 1: finishing
11:10:05: Main    : thread 1 done
11:10:05: Main    : before joining thread 2.
11:10:05: Main    : thread 2 done
